In [195]:
import numpy as np
import pandas as pd
import os
import json

In [217]:

def DataAggregator(path):
    path ='E:/CESC621/Final Project/pan18-style-change-detection-test-dataset-2018-01-31/'
    text_dict = {}
    json_dict = {}
    label_df = pd.DataFrame(columns=['changes', 'positions','file'])

    for file in os.listdir(path):
        if file.endswith('.txt'):
            with open(path+file, 'r', encoding='utf-8') as myfile:
                text_dict[file.replace('problem-', '').replace('.txt', '')] = myfile.read().replace('\n', '')
        else:
            with open(path+file, 'r', encoding='utf-8') as myfile:
                data = json.load(myfile)
                json_dict[file.replace('problem-', '').replace('.truth', '')] = data['changes']

    df = pd.DataFrame(list(text_dict.items()), columns=['file', 'text'])
    label_df = pd.DataFrame(list(json_dict.items()), columns = ['file', 'changes'])

    test_df = df.merge(label_df,on='file')
    return test_df

In [218]:
df = DataAggregator('E:/CESC621/Final Project/pan18-style-change-detection-test-dataset-2018-01-31/')

In [219]:
df.head()

,file,text,changes
0,1,If we look at the Agile fluency model. The Scr...,True
1,10,"You can estimate a completion date, date range...",True
2,100,This item works like the Fast TM but it teache...,True
3,1000,More popular party (on the left wing) was Soci...,False
4,1001,Note that the spiritual precursor to the Magna...,True
